In [ ]:
# Download the dataset from github (Original dataset)
# !pip install wget
# import wget
# import zipfile

# wget.download('https://github.com/jbrownlee/Datasets/releases/download/Flickr8k/Flickr8k_text.zip')
# wget.download('https://github.com/jbrownlee/Datasets/releases/download/Flickr8k/Flickr8k_Dataset.zip')


# zip_flicker_dataset = '/content/drive/MyDrive/Data/Flickr8k_Dataset.zip'
# zip_flicker_text = '/content/drive/MyDrive/Data/Flickr8k_text.zip'

# with zipfile.ZipFile(zip_flicker_dataset) as zip:
#   zip.extractall('/content/drive/MyDrive/Data/Image-Captioning-Data/Flicker-Data')
# with zipfile.ZipFile(zip_flicker_text) as zip:
#   zip.extractall('/content/drive/MyDrive/Data/Image-Captioning-Data/Flicker-Text')

In [ ]:
from keras.preprocessing import image
from keras.applications.vgg16 import VGG16, preprocess_input
from keras.models import Model
from keras.layers import Flatten
from PIL import Image
from tqdm import tqdm
import os
import pickle
import matplotlib.pyplot as plt
import tensorflow as tf

IMAGE_SIZE = 224
IMAGE_DATA_DIR = '/content/drive/MyDrive/Data/Image-Captioning-Data/Flicker-Data/Flicker8k_Dataset'

In [ ]:
base_model = VGG16(include_top=True)

for i in range(len(base_model.layers)):
    curr_layer = base_model.layers[i]
    if 'conv' not in curr_layer.name:
        print(curr_layer.name)
        # to get config print -> curr_layer.get_config()
        continue
    
    filters, biasis = curr_layer.get_weights()
    
    print(i, curr_layer.name, filters.shape)

553476096/553467096 [==============================] - 4s 0us/step
input_1
1 block1_conv1 (3, 3, 3, 64)
2 block1_conv2 (3, 3, 64, 64)
block1_pool
4 block2_conv1 (3, 3, 64, 128)
5 block2_conv2 (3, 3, 128, 128)
block2_pool
7 block3_conv1 (3, 3, 128, 256)
8 block3_conv2 (3, 3, 256, 256)
9 block3_conv3 (3, 3, 256, 256)
block3_pool
11 block4_conv1 (3, 3, 256, 512)
12 block4_conv2 (3, 3, 512, 512)
13 block4_conv3 (3, 3, 512, 512)
block4_pool
15 block5_conv1 (3, 3, 512, 512)
16 block5_conv2 (3, 3, 512, 512)
17 block5_conv3 (3, 3, 512, 512)
block5_pool
flatten
fc1
fc2
predictions


In [ ]:
# For extracting every layers output in vgg16
features_list = [layer.output for layer in base_model.layers]
feat_extraction_model = tf.keras.Model(inputs=base_model.input, outputs=features_list)

In [ ]:
print(base_model.layers[-2].output)

KerasTensor(type_spec=TensorSpec(shape=(None, 4096), dtype=tf.float32, name=None), name='fc2/Relu:0', description="created by layer 'fc2'")


In [ ]:
base_model = VGG16(include_top=True)
base_model.trainable = False

# Taking till Flaten layer
model = Model(inputs = base_model.inputs, outputs = base_model.layers[-2].output)

img = image.img_to_array(image.load_img('/content/drive/MyDrive/Data/Image-Captioning-Data/Flicker-Data/Flicker8k_Dataset/35506150_cbdb630f4f.jpg', target_size=(224, 224)))
img = img.reshape(-1, 224, 224, 3)
img = preprocess_input(img)

# model.summary()
img_features = model.predict(img)
base_model.summary()

Model: "vgg16"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 224, 224, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 224, 224, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 112, 112, 64)      0         
                                                                 
 block2_conv1 (Conv2D)       (None, 112, 112, 128)     73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 112, 112, 128)     147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 56, 56, 128)       0     

In [ ]:
# (no. of samples, features)
img_features.shape

(1, 4096)

## Feature extraction

In [ ]:
# features = {}

# for full_img_name in tqdm(os.listdir(IMAGE_DATA_DIR)):
#     # removing image format from image name
#     img_name = full_img_name.split('.')[0]

#     img_dir = os.path.join(IMAGE_DATA_DIR, full_img_name)

#     # loading image and converting to numpy array and reshaping it to (224, 224, 3)
#     # Image size is 224 because thats the image size in which vgg16 is trained on
#     loaded_img = image.img_to_array(image.load_img(img_dir, target_size=(224, 224))).reshape(-1, 224, 224, 3)
    
#     #preprocessing data according to vgg16
#     loaded_img = preprocess_input(loaded_img)

#     features[img_name] = model.predict(loaded_img)

## Saving features and model

In [ ]:
# Saving features
# pickle.dump(features, open('/content/drive/MyDrive/Data/Image-Captioning-Data/features.pkl', 'wb'))

In [ ]:
# pickle.dump(model, open('/content/drive/MyDrive/Data/Image-Captioning-Data/model.h5', 'wb'))

In [ ]:
mx_caption_length = 0
def load_caption_data(filepath):
    caption_mapping = {}
    global mx_caption_length

    # Reading caption file
    with open(filepath, 'r') as file:
        captions = file.read()

    #looping through every line
    for line in captions.split('\n'):
        #spliting b/w captions and image name
        split_line = line.split()
        if len(split_line) <= 3:
            print(split_line)
            continue
        # fetching image name without format
        img_name = split_line[0].split('.')[0]
        # fetching caption
        curr_caption = ' '.join(split_line[1:])
        mx_caption_length = max(mx_caption_length, len(split_line))

        # if img_name key is not in dict then declared that key
        if img_name not in caption_mapping:
            caption_mapping[img_name] = []
        # appending every caption for an img_name
        caption_mapping[img_name].append(curr_caption)
    
    return caption_mapping

In [ ]:
load_caption_data('/content/drive/MyDrive/Data/Image-Captioning-Data/Flicker-Text/Flickr8k.token.txt')

['2165461920_1a4144eb2b.jpg#0', 'dogs', 'racing']
['2428275562_4bde2bc5ea.jpg#0', 'A']
['244760301_5809214866.jpg#3', 'People', 'walking']
['256085101_2c2617c5d0.jpg#3', 'Dog', 'yawns']
['2714703706_d21c5cb8df.jpg#0', 'dogs', 'playing']
['2755314937_1e974bf2b5.jpg#3', 'broken', 'image']
['2862481071_86c65d46fa.jpg#4', 'Trucks', 'racing']
['2929669711_b2d5a640f0.jpg#4', 'man', 'surfing']
['3108732084_565b423162.jpg#2', 'a', 'snowboarder']
['3125309108_1011486589.jpg#2', 'rugby', 'match']
['3154693053_cfcd05c226.jpg#0', 'A', 'basketball']
['3189251454_03b76c2e92.jpg#3', 'dog', 'barking']
['3237760601_5334f3f3b5.jpg#1', 'A', 'skier']
['3360823754_90967276ec.jpg#3', 'Man', 'skateboarding']
['3640443200_b8066f37f6.jpg#0', 'a']
['3664928753_7b0437fedf.jpg#3', 'Javelin', 'competition']
['3694071771_ce760db4c7.jpg#0', 'a', 'cyclist']
[]


39

In [ ]:
base_model_out = base_model.output
base_model_out = tf.keras.layers.Reshape((-1, base_model_out.shape[-1]))(base_model_out)

In [ ]:
model = tf.keras.Model(base_model.input, base_model_out)

In [ ]:
model.summary()

Model: "model_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 224, 224, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 224, 224, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 112, 112, 64)      0         
                                                                 
 block2_conv1 (Conv2D)       (None, 112, 112, 128)     73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 112, 112, 128)     147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 56, 56, 128)       0   